In [1]:
from main import *
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import missingno as msno
import numpy as np
from datetime import datetime, timedelta
from shapely.geometry import MultiPoint
from concurrent.futures import ThreadPoolExecutor

sns.set_style('whitegrid')

In [ ]:
def parallel_requests(start_date, end_date):
    # Intervalo de tempo entre as solicitações (1 hora)
    hour_interval = timedelta(hours=1)
    density_df= pd.DataFrame()
    results = []
    with ThreadPoolExecutor(max_workers=500) as executor:  # Escolha o número adequado de workers
        while start_date <= end_date:
            initial_date = (start_date - timedelta(seconds=0))
            initial_date = str(initial_date)+'.000'
            final_date =( start_date + timedelta(seconds=3))
            final_date = str(final_date)+'.999'

            df_cat62 = executor.submit(request_api, 'cat-62', initial_date, final_date).result()
            #results.append(executor.submit(request_api, 'cat-62', initial_date, final_date))
            try:
   
                coord_pairs_formatted = [f"({lat} {lon})" for lat, lon in zip(df_cat62['lat'], df_cat62['lon'])]

                multipoint_wkt = f"MULTIPOINT ({', '.join(coord_pairs_formatted)})"
            except:
                multipoint_wkt = np.NaN


            print(start_date)

            density_df_aux= pd.DataFrame({'Date/Time':[start_date],'MULTIPOINT':[multipoint_wkt]})

            # Exibir a representação WKT resultante
            density_df = pd.concat([density_df, density_df_aux], ignore_index=True)
            start_date += hour_interval

    
    return density_df

In [8]:
def parallel_requests(start_date, end_date):
    # Intervalo de tempo entre as solicitações (1 hora)

    density_df= pd.DataFrame()

    start_date = str(start_date)+'.000'
    end_date = str(end_date)+'.000'
    
    with ThreadPoolExecutor(max_workers=500) as executor:  # Escolha o número adequado de workers
            df_cat62 = executor.submit(request_api, 'cat-62', start_date, end_date).result()

    
    return df_cat62

In [9]:
# Dataframe principal onde você vai salvar os resultados
df_main = pd.DataFrame()


# Defina a data inicial e final para o loop
start_date = datetime(2022, 6, 1, 0, 0, 0)
end_date = datetime(2022, 6, 10, 0, 0, 0)

# Realize as solicitações em paralelo e colete os resultados
results = parallel_requests(start_date, end_date)

results

,flightid,lat,lon,flightlevel,speed,dt_radar
0,06cee9cc4f09c777a6fbc656d1484547,-0.522816,-0.891742,NaN,103.0,1654365903000
1,06cee9cc4f09c777a6fbc656d1484547,-0.522783,-0.891726,NaN,103.0,1654365904000
2,9dcb2085c3fbba608f5b447b9f4d93f9,-0.478124,-0.871190,210.0,252.0,1654225560000
3,9dcb2085c3fbba608f5b447b9f4d93f9,-0.463469,-0.863726,210.0,251.0,1654226344000
4,9dcb2085c3fbba608f5b447b9f4d93f9,-0.475829,-0.870013,210.0,251.0,1654225681000
...,...,...,...,...,...,...
714187,5026d4a528507da670372707bd131118,-0.400626,-0.749134,40.0,168.0,1654179063000
714188,5026d4a528507da670372707bd131118,-0.401292,-0.746617,55.0,212.0,1654178826000
714189,5026d4a528507da670372707bd131118,-0.399344,-0.753385,5.0,118.0,1654179484000
714190,5026d4a528507da670372707bd131118,-0.401992,-0.746932,55.0,186.0,1654178882000


In [13]:
data_mean = results[['flightid','speed']].groupby('flightid').mean()
data_mean['speed'] =data_mean['speed'] *1.61
data_mean

,speed
flightid,
0003d070d3cddc8a27e29b0d7ec5d22e,448.903069
001e4e187357822246184b8dd59b7656,531.960100
0024acd89f59723c23813e1f7a0156f3,457.804330
00300609f06f0d13f08f88f17dac6fd6,671.315191
00338c0abbbf5eb2106a2200406e9c65,478.218060
...,...
ffd651426793f951350d7e4924e4485a,527.041974
ffe2486cbea0418b18534646a77b6c35,510.727778
ffeb8d1dc534c50db3f04f6614a8d718,498.066716


In [ ]:
def parallel_requests(start_date, end_date):
    # Intervalo de tempo entre as solicitações (1 hora)
    hour_interval = timedelta(hours=1)
    density_df= pd.DataFrame()
    results = []
    with ThreadPoolExecutor(max_workers=500) as executor:  # Escolha o número adequado de workers
        while start_date <= end_date:
            initial_date = (start_date - timedelta(seconds=0))
            initial_date = str(initial_date)+'.000'
            final_date =( start_date + timedelta(seconds=3))
            final_date = str(final_date)+'.999'

            df_cat62 = executor.submit(request_api, 'cat-62', initial_date, final_date).result()
            
            try:
   
                coord_pairs_formatted = [f"({lat} {lon})" for lat, lon in zip(df_cat62['lat'], df_cat62['lon'])]

                multipoint_wkt = f"MULTIPOINT ({', '.join(coord_pairs_formatted)})"
            except:
                multipoint_wkt = np.NaN

            density_df_aux= pd.DataFrame({'Date/Time':[start_date],'MULTIPOINT':[multipoint_wkt]})

            # Exibir a representação WKT resultante
            density_df = pd.concat([density_df, density_df_aux], ignore_index=True)
            start_date += hour_interval

    
    return density_df

In [34]:
# Dataframe principal onde você vai salvar os resultados
df_main = pd.DataFrame()


# Defina a data inicial e final para o loop
start_date = datetime(2022, 6, 1, 0, 0, 0)
end_date = datetime(2022, 7, 1, 0, 0, 0)

# Intervalo de tempo entre as solicitações (1 hora)
hour_interval = timedelta(hours=1)

# Realize as solicitações em paralelo e colete os resultados
results = parallel_requests(start_date, end_date)


    
results

2022-06-01 00:00:00
2022-06-01 01:00:00
2022-06-01 02:00:00
2022-06-01 03:00:00
2022-06-01 04:00:00
2022-06-01 05:00:00
2022-06-01 06:00:00
2022-06-01 07:00:00
2022-06-01 08:00:00
2022-06-01 09:00:00
2022-06-01 10:00:00
2022-06-01 11:00:00
2022-06-01 12:00:00
2022-06-01 13:00:00
2022-06-01 14:00:00
2022-06-01 15:00:00
2022-06-01 16:00:00
2022-06-01 17:00:00
2022-06-01 18:00:00
2022-06-01 19:00:00
2022-06-01 20:00:00
2022-06-01 21:00:00
2022-06-01 22:00:00
2022-06-01 23:00:00
2022-06-02 00:00:00
2022-06-02 01:00:00
2022-06-02 02:00:00
2022-06-02 03:00:00
2022-06-02 04:00:00
2022-06-02 05:00:00
2022-06-02 06:00:00
2022-06-02 07:00:00
2022-06-02 08:00:00
2022-06-02 09:00:00
2022-06-02 10:00:00
2022-06-02 11:00:00
2022-06-02 12:00:00
2022-06-02 13:00:00
2022-06-02 14:00:00
2022-06-02 15:00:00
2022-06-02 16:00:00
2022-06-02 17:00:00
2022-06-02 18:00:00
2022-06-02 19:00:00
2022-06-02 20:00:00
2022-06-02 21:00:00
2022-06-02 22:00:00
2022-06-02 23:00:00
2022-06-03 00:00:00
2022-06-03 01:00:00


,Date/Time,MULTIPOINT
0,2022-06-01 00:00:00,NaN
1,2022-06-01 01:00:00,"MULTIPOINT ((-0.4045450401 -0.8040536104), (-0..."
2,2022-06-01 02:00:00,"MULTIPOINT ((-0.4542044599 -0.8557811118), (-0..."
3,2022-06-01 03:00:00,"MULTIPOINT ((-0.4044566834 -0.7817160046), (-0..."
4,2022-06-01 04:00:00,"MULTIPOINT ((-0.2658154075 -0.7027435313), (-0..."
...,...,...
716,2022-06-30 20:00:00,"MULTIPOINT ((-0.4126228945 -0.8110181005), (-0..."
717,2022-06-30 21:00:00,NaN
718,2022-06-30 22:00:00,"MULTIPOINT ((-0.3204977765 -0.710032636), (-0...."
719,2022-06-30 23:00:00,"MULTIPOINT ((-0.4081511538 -0.8052053781), (-0..."


In [36]:
results.to_csv('density.csv')